In [1]:
from openai import OpenAI
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.beta.assistant_stream_event import ThreadMessageDelta
from openai.types.beta.threads.text_delta_block import TextDeltaBlock 
import os
import pandas as pd
from tqdm import tqdm
import pickle

In [2]:
df_scrape = pd.read_pickle('/app/Store/dataframe.pkl')
df_asst = pd.read_csv('/app/Store/assistant_files.csv')

df_asst.rename(columns={'filename': 'file_path'}, inplace=True)

# Merging on the common column 'key'
df_merge = pd.merge(df_scrape, df_asst, on='file_path', how='inner')

In [3]:
import os
import glob
from tqdm import tqdm
from openai import OpenAI

# Initialize your client
client = OpenAI()

# Define folder path
folder_path = '/app/UO Wellness Classes/'

# Extend the path to include only .txt files
txt_files = glob.glob(os.path.join(folder_path, '*.txt'))


In [4]:


file_ids = []

# Iterate through the text files and upload
for txt_file in tqdm(txt_files):
    try:
        with open(txt_file, 'rb') as file:
            upload_file = client.files.create(
                file=file,
                purpose='assistants'
            )
            file_ids.append(upload_file.id)
    except Exception as e:
        print(f"Error uploading file {txt_file}: {e}")


100%|██████████| 11/11 [00:06<00:00,  1.63it/s]


In [14]:
with open('/app/class_file_ids.pkl', 'wb') as file:
    pickle.dump(file_ids,file)

In [5]:
with open('/app/class_file_ids.pkl', 'rb') as file:
   file_ids = pickle.load(file)

In [9]:
vector_store_id = 'vs_67dfa578a634819189a0fa80b0f5b470'
# Process each uploaded file
for file_id in tqdm(file_ids):
    try:
        batch = client.vector_stores.files.create(
            vector_store_id=vector_store_id,
            file_id=file_id
        )
    except Exception as e:
        print(f"Error processing file with ID {file_id}: {e}")

100%|██████████| 11/11 [00:08<00:00,  1.25it/s]


In [10]:
list_of_dicts = []
for text_file, file_id in zip(txt_files,file_ids):

    entry = {
        "url":"https://catalog.uoregon.edu/courses/" ,
        "text_hash":"",
        "date_accessed": "", 
        "file_path": text_file,
        "file_id": file_id,
        "vectorstore_id": vector_store_id , 
        "assistant_id": "" ,
    }

    list_of_dicts.append(entry)

classes_df = pd.DataFrame(list_of_dicts)

In [12]:
classes_df.to_csv('/app/appdata/classes.csv', index=False)